In [1]:
import pandas as pd
import mne as mne
import os 
import time
import numpy as np
import matplotlib.pyplot as plt
import joblib
import constants
from IPython.utils import io
import time
import sys
import yasa
from scipy.signal import welch

#Import my modules
import format_eeg_data
import constants
import eeg_stat_ts

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))
/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package yasa is out of date. Your version is 0.6.3, the latest is 0.6.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [49]:
data_types = ['N2', 'N3','REM', 'N1', 'Wake']
data_types = ['Wake','N3','REM', 'N1','N2']

for data_type in data_types:
    
    # 1. generate all path names and class list(s) etc. 
    folder = '/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/'
    paths = joblib.load(folder + data_type + '_paths.pkl')
    ts_df = pd.read_hdf('Multiple_PSDs_Per_Participant/' + data_type + 'multiple_psds_per_person.h5' , key = 'df')
    
    groups = pd.Series(paths['s_p_id'])
    s_class_list = pd.Series(paths['s_class_list'])
    y = s_class_list.map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3}) 
    
    ts_df['y'] = y 
    ts_df['groups'] = groups
    
    final_df = {}
    for col in list(ts_df.columns):
        final_df[col] = []
    
    for ind in ts_df.index:
        row = ts_df.loc[ind]
        for col in ts_df.columns:
            if col in ['y','groups']:
                series = row['39.0_Right Temporal'] #this is to get the number of elements to populate y and groups vector with
                val = row[col]
                final_df[col]+= [val for i in range(len(series)) ]
            else:
                series = row[col]
                final_df[col] += list(series.values)
    
    final_df = pd.DataFrame(final_df)
    final_df.to_hdf(data_type + 'processed_multiple_psds_per_person.h5' , key = 'df', mode = 'w')
    print(data_type + '  done!')

Wake  done!
N3  done!
REM  done!
N1  done!
N2  done!
